---
title: Exploring CHIRPS data
short_title: Exploring CHIRPS data
---

Load data from NetCDF using [earthkit](https://ecmwf.github.io/earthkit-website/) and aggregate the data to DHIS2 organization units.  

In [23]:
import earthkit

Load a NetCDF file using earthkit. See [more examples](https://earthkit-data.readthedocs.io/en/latest/examples/) for how you can load data with eartkit. 

In [21]:
file = "data/chirps-v3.0.2024.monthly.nc"
# file = "data/pm_final_srilanka_linearp.nc"
# file = "data/era5-land-daily-mean-temperature-2m-july-2025-sierra-leone.nc"
data = earthkit.data.from_source("file", file)

# xarray.open_dataset(file)

To display the contents of the dataset we can convert it to an [xarray](https://xarray.dev). It shows that the file includes 3 dimensions (latitude, longitude and valid_time) and one data variable "precip" (precipitation). The data source is CHIRPS Version 3.0 by Climate Hazards Center.

In [24]:
data.to_xarray()

<xarray.Dataset> Size: 829MB
Dimensions:    (latitude: 2400, longitude: 7200, time: 12)
Coordinates:
  * latitude   (latitude) float32 10kB -59.97 -59.92 -59.88 ... 59.92 59.97
  * longitude  (longitude) float32 29kB -180.0 -179.9 -179.9 ... 179.9 180.0
  * time       (time) datetime64[ns] 96B 2024-01-01T00:01:16.933431296 ... 20...
Data variables:
    precip     (time, latitude, longitude) float32 829MB dask.array<chunksize=(1, 400, 1200), meta=np.ndarray>
Attributes: (12/15)
    Conventions:       CF-1.6
    title:             CHIRPS Version 3.0
    history:           created by Climate Hazards Center
    version:           Version 3.0
    data_created:      2025-02-25
    creator_name:      Pete Peterson
    ...                ...
    reference:         Funk, C.C., Peterson, P.J., Landsfeld, M.F., Pedreros,...
    comments:           time variable denotes the first day of the given month. 
    acknowledgements:  The Climate Hazards Center InfraRed Precipitation with...
    ftp_url:           ftp://chg-ftpout.geog.ucsb.edu/pub/chg/products/CHIRPS...
    website:           http://chg.geog.ucsb.edu/data/chirps/index.html
    faq:               http://chg-wiki.geog.ucsb.edu/wiki/CHIRPS_FAQ

eartkit can also be used to load the organisation units from DHIS2 that we [saved as a GeoJSON file](organization-units). 

In [4]:
district_file = "data/sierra-leone-districts.geojson"
features = earthkit.data.from_source("file", district_file)

We can display the first feature to see the information we have for each org unit. For the aggregation, we are particularly interested in the id and the geometry (polygon) of the org unit. 

In [5]:
features[:1]

[type                                                                Polygon
 id                                                              O6uvpzGd5pu
 name                                                                     Bo
 hasCoordinatesDown                                                     True
 hasCoordinatesUp                                                      False
 level                                                                     2
 grandParentParentGraph                                                     
 grandParentId                                                              
 parentGraph                                                     ImspTQPwCqd
 parentId                                                        ImspTQPwCqd
 parentName                                                     Sierra Leone
 dimensions                                                              { }
 weight                                                                    1

To aggregate the data to the org unit features we use the aggregate package of [earthkit-transforms](https://earthkit-transforms.readthedocs.io). We keep the daily period type and only aggregate the data spatially to the org unit features. mask_dim is the dimension (org unit id) that will be created after the reduction of the spatial dimensions (longitude/latitude grid). 

In [6]:
agg_data = aggregate.spatial.reduce(data, features, mask_dim="id")

The aggregated data is returned as an xarray with two dimensions (id and valid_time), and the same temperature vaiable. 

In [7]:
agg_data

<xarray.Dataset> Size: 2kB
Dimensions:     (valid_time: 31, id: 13)
Coordinates:
    number      int64 8B 0
  * valid_time  (valid_time) datetime64[ns] 248B 2025-07-01 ... 2025-07-31
  * id          (id) object 104B 'O6uvpzGd5pu' 'fdc6uOvgoji' ... 'at6UHUQatSo'
Data variables:
    t2m         (id, valid_time) float32 2kB 296.8 297.3 297.3 ... 298.6 298.1
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-08-07T14:32 GRIB to CDM+CF via cfgrib-0.9.1...

Next, we select the variable we would like to import to DHIS2 (t2m). We also convert temperatures in kelvin to celcius by subtracting 273.15 from the values.

In [8]:
data_array = agg_data['t2m'] - 273.15

This returns a multidimensional xarray.dataArray with the two dimensions (id and valid_time). Two decimals is sufficient for our use so we round all the temperature values:  

In [9]:
rounded_array = data_array.astype('float64').round(decimals = 2)

The two dimensions are "stacked" into one dimension using the [xarray stack method](https://docs.xarray.dev/en/latest/generated/xarray.DataArray.stack.html): 

In [10]:
stacked_array = rounded_array.stack(index=[...])

Use the `data_array_to_dhis2_json` to translate the data array into JSON used by DHIS2 Web API:

In [11]:
json_dict = data_array_to_dhis2_json(
  data_array = stacked_array,     # stacked data array
  org_unit_dim = 'id',            # dimension containing the org unit id
  period_dim = 'valid_time',      # dimension containing the period 
  data_element_id= 'VJwwPOOvge6'  # id of the DHIS2 data element
);

We can display the first 3 items to see that we have one temperatue value for each org unit and period combination.

In [19]:
json_dict['dataValues'][:3]

[{'dataElement': 'VJwwPOOvge6',
  'orgUnit': 'O6uvpzGd5pu',
  'period': '20250701',
  'value': '23.68'},
 {'dataElement': 'VJwwPOOvge6',
  'orgUnit': 'fdc6uOvgoji',
  'period': '20250701',
  'value': '23.96'},
 {'dataElement': 'VJwwPOOvge6',
  'orgUnit': 'lc3eMKXaEfw',
  'period': '20250701',
  'value': '24.52'}]